# Momi2 analysis
## Notebook must be run in python3

In [428]:
import momi
import glob
import numpy as np
import pandas as pd
import ipyparallel as ipp
import datetime
import time
import logging
import sys
import os

import warnings
warnings.filterwarnings('ignore')

simout = "simout"
popfile = "{}/simpops.txt".format(simout)

logging.basicConfig(level=logging.DEBUG,
                    filename="momi_log.txt")

In [422]:
def progressbar(njobs, finished, msg="", spacer="  "):
    """ prints a progress bar """
    if njobs:
        progress = 100*(finished / float(njobs))
    else:
        progress = 100

    hashes = '#'*int(progress/5.)
    nohash = ' '*int(20-len(hashes))

    args = [spacer, hashes+nohash, int(progress), msg]
    print("\r{}[{}] {:>3}% {} ".format(*args), end="")
    sys.stdout.flush()
for i in range(100):
    progressbar(100, i, "watdo")
progressbar(100, 100, "done")

  [####################] 100% done  

## Generate the popsfile
For 2 population models just split the vcf samples in half and assign the first
half to pop1, and the second half to pop2

In [67]:
dat = !head simout/nomigration-sim0.vcf 
## Relying on the fact that the vcf files 5th line contains the samples
## which are indexed as 9 and on
samps = dat[5].split()[9:]
print(samps)
samps_per_pop = int(len(samps)/2)
popdict = {x:"pop1" for x in samps[:samps_per_pop]}
popdict.update({x:"pop2" for x in samps[samps_per_pop:]})
print(popdict)
with open(popfile, 'w') as outfile:
    for k,v in popdict.items():
        outfile.write("{}\t{}\n".format(k,v))
!cat simout/simpops.txt

['msp_0', 'msp_1', 'msp_2', 'msp_3', 'msp_4', 'msp_5', 'msp_6', 'msp_7']
{'msp_0': 'pop1', 'msp_1': 'pop1', 'msp_2': 'pop1', 'msp_3': 'pop1', 'msp_4': 'pop2', 'msp_5': 'pop2', 'msp_6': 'pop2', 'msp_7': 'pop2'}
msp_0	pop1
msp_1	pop1
msp_2	pop1
msp_3	pop1
msp_4	pop2
msp_5	pop2
msp_6	pop2
msp_7	pop2


## Generate sfs files per vcf
* **Population assignment file** - This is a tab or space separated list of sample names and population names to which they are assigned. Sample names need to be exactly the same as they are in the VCF file. Population names can be anything, but it’s useful if they’re meaningful.  
* **Properly formatted VCF** - We do have the VCF file output from the ipyrad Anolis assembly, but it requires a bit of massaging before it’s ready for momi2. It must be zipped and indexed in such a way as to make it searchable.  
* **BED file** - This file specifies genomic regions to include in when calculating the SFS. It is composed of 3 columns which specify ‘chrom’, ‘chromStart’, and ‘chromEnd’.  
* **The allele counts file** - The allele counts file is an intermediate file that we must generate on the way to constructing the SFS. momi2 provides a function for this.  
* **Genereate the SFS** - The culmination of all this housekeeping is the SFS file which we will use for demographic inference.

In [550]:
%%bash
for i in `ls simout/*/*.vcf`; \
    do echo $i; \
    bgzip -c $i > $i.gz; \
    tabix $i.gz;\
    python vcf2bed.py $i;
    done

simout/asymmetric_migration/asymmetric_migration-sim0.vcf
simout/asymmetric_migration/asymmetric_migration-sim1.vcf
simout/asymmetric_migration/asymmetric_migration-sim2.vcf
simout/asymmetric_migration/asymmetric_migration-sim3.vcf
simout/asymmetric_migration/asymmetric_migration-sim4.vcf
simout/no_migration/no_migration-sim0.vcf
simout/no_migration/no_migration-sim1.vcf
simout/no_migration/no_migration-sim2.vcf
simout/no_migration/no_migration-sim3.vcf
simout/no_migration/no_migration-sim4.vcf
simout/symmetric_migration/symmetric_migration-sim0.vcf
simout/symmetric_migration/symmetric_migration-sim1.vcf
simout/symmetric_migration/symmetric_migration-sim2.vcf
simout/symmetric_migration/symmetric_migration-sim3.vcf
simout/symmetric_migration/symmetric_migration-sim4.vcf


In [551]:
model_dirs = {x.split("/")[1]:x for x in glob.glob("simout/*") if os.path.isdir(x)}
sfs_dict = {x:[] for x in model_dirs}
print(model_dirs)
print(sfs_dict)

for m, d in model_dirs.items():
    for f in glob.glob(d + "/*.vcf.gz"):
        print(f)
        allele_counts = momi.SnpAlleleCounts.read_vcf(vcf_file=f, ind2pop=popdict, bed_file=f.split(".")[0]+".bed", ancestral_alleles=False)
        allele_counts.dump(f.split(".")[0]+"_allele_counts.gz")
        sfs = allele_counts.extract_sfs(n_blocks=None)
        sfs_dict[m].append(sfs)
        sfs.dump(f.split(".")[0]+".sfs")
print(len(sfs_dict))

{'no_migration': 'simout/no_migration', 'asymmetric_migration': 'simout/asymmetric_migration', 'symmetric_migration': 'simout/symmetric_migration'}
{'no_migration': [], 'asymmetric_migration': [], 'symmetric_migration': []}
simout/no_migration/no_migration-sim1.vcf.gz
simout/no_migration/no_migration-sim4.vcf.gz
simout/no_migration/no_migration-sim2.vcf.gz
simout/no_migration/no_migration-sim3.vcf.gz
simout/no_migration/no_migration-sim0.vcf.gz
simout/asymmetric_migration/asymmetric_migration-sim1.vcf.gz
simout/asymmetric_migration/asymmetric_migration-sim0.vcf.gz
simout/asymmetric_migration/asymmetric_migration-sim3.vcf.gz
simout/asymmetric_migration/asymmetric_migration-sim2.vcf.gz
simout/asymmetric_migration/asymmetric_migration-sim4.vcf.gz
simout/symmetric_migration/symmetric_migration-sim1.vcf.gz
simout/symmetric_migration/symmetric_migration-sim4.vcf.gz
simout/symmetric_migration/symmetric_migration-sim3.vcf.gz
simout/symmetric_migration/symmetric_migration-sim2.vcf.gz
simout/sym

In [258]:
def optimize(sfs):
    no_migration_model = momi.DemographicModel(N_e=1e5,
                                               #muts_per_gen=1e-8,
                                               gen_time=1)

    no_migration_model.add_time_param("tdiv")

    no_migration_model.add_leaf("pop1")
    no_migration_model.add_leaf("pop2")
    no_migration_model.move_lineages("pop1", "pop2", t="tdiv")

    no_migration_model.set_data(sfs)

    return no_migration_model.optimize()
results = []
for sfs in sfs_list:
    results.append(optimize(sfs))
print(np.mean([x.parameters.tdiv for x in results]))
print(min([x.parameters.tdiv for x in results]), max([x.parameters.tdiv for x in results]))

/home/isaac/miniconda2/envs/momi-py36/lib/python3.6/site-packages/momi/compute_sfs.py:511: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  [slice(None)]],
/home/isaac/miniconda2/envs/momi-py36/lib/python3.6/site-packages/autograd/numpy/numpy_boxes.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  def __getitem__(A, idx): return A[idx]
/home/isaac/miniconda2/envs/momi-py36/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:597: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)

200156.892920796
187080.75895088378 210358.50332806082


In [470]:
def optimize(sfs, test_model="no_migration"):
    import momi
    import pandas as pd
    model = momi.DemographicModel(N_e=1e5,
                                  #muts_per_gen=1e-8,
                                  gen_time=1)

    model.add_leaf("pop1")
    model.add_leaf("pop2")

    ## Move some lineages one way or the other or both
    if test_model == "asymmetric_migration":
        model.add_time_param("tmig_pop1_pop2")
        model.add_time_param("tmig_pop2_pop1")
        model.add_pulse_param("mfrac_pop1_pop2", upper=.2)
        model.add_pulse_param("mfrac_pop2_pop1", upper=.2)
        model.move_lineages("pop1", "pop2", t="tmig_pop1_pop2", p="mfrac_pop1_pop2")
        model.move_lineages("pop2", "pop1", t="tmig_pop1_pop2", p="mfrac_pop2_pop1")
        model.add_time_param("tdiv", lower_constraints=["tmig_pop1_pop2", "tmig_pop2_pop1"])

    elif test_model == "symmetric_migration":
        model.add_pulse_param("mfrac", upper=.2)
        model.add_time_param("tmig")
        model.move_lineages("pop1", "pop2", t="tmig", p="mfrac")
        model.move_lineages("pop2", "pop1", t="tmig", p="mfrac")
        model.add_time_param("tdiv", lower_constraints=["tmig"])
    else:
        ## No migration
        model.add_time_param("tdiv")

    ## Divergence event
    model.move_lineages("pop1", "pop2", t="tdiv")

    model.set_data(sfs)

    return pd.Series(model.optimize())
#results = []
#for sfs in sfs_list:
#    results.append(optimize(sfs))

## Run optimization for each model serially
Slow

In [406]:
models = ["no_migration", "symmetric_migration", "asymmetric_migration"]
models = sfs_dict.keys()
## Results dictionary for keeping track ouf output testing each model on the simulated data
res_dict = {x:{y:pd.DataFrame() for y in models} for x in models}
for model in models:
    for testmodel in models:
        print(model, testmodel)
        for i, sfs in enumerate(sfs_dict[model]):
            print(i),
            res = pd.Series(optimize(sfs, test_model=testmodel))
            res_dict[model][testmodel] = res_dict[model][testmodel].append(res, ignore_index=True)
res_dict["no_migration"]["no_migration"]["log_likelihood"]

no_migration no_migration
0
1
2
3
4


## Run optimization for each model in parallel
100 loci runs in 30'

In [580]:
## Run this in a python3 env and wait for clients to attach
#!ipcluster start -n 4 --cluster-id=ipp3 --daemonize
ipyclient = ipp.Client(cluster_id="ipp3")
print(len(ipyclient.ids))

5


In [566]:
ipyclient[:].use_dill()
lbview = ipyclient.load_balanced_view()

start = time.time()
printstr = " |  {}  |  {}/{}  "
res_dict = {x:{y:pd.DataFrame() for y in models} for x in models}
for model in models:
    for testmodel in models:
        parallel_jobs = {}
        for i, sfs in enumerate(sfs_dict[model]):
            parallel_jobs[i] = lbview.apply(optimize, *(sfs, testmodel))

        ## Wait for all jobs to finish
        while 1:
            fin = [i.ready() for i in parallel_jobs.values()]
            elapsed = datetime.timedelta(seconds=int(time.time()-start))
            progressbar(len(fin), sum(fin), printstr.format(elapsed, model, testmodel))
            time.sleep(0.1)
            if len(fin) == sum(fin):
                print("")
                break

        ## Fetch results
        for job in parallel_jobs:
            if parallel_jobs[job].ready():
                if not parallel_jobs[job].successful():
                    raise Exception((job, parallel_jobs[job].result()))
                else:
                    res = parallel_jobs[job].result()
                    res_dict[model][testmodel] = res_dict[model][testmodel].append(res, ignore_index=True)


  [####################] 100%  |  0:00:08  |  no_migration/no_migration   
  [####################] 100%  |  0:04:46  |  no_migration/asymmetric_migration   
  [####################] 100%  |  0:09:18  |  no_migration/symmetric_migration   
  [####################] 100%  |  0:09:31  |  asymmetric_migration/no_migration   
  [####################] 100%  |  0:15:29  |  asymmetric_migration/asymmetric_migration   
  [####################] 100%  |  0:21:59  |  asymmetric_migration/symmetric_migration   
  [####################] 100%  |  0:22:10  |  symmetric_migration/no_migration   
  [####################] 100%  |  0:29:57  |  symmetric_migration/asymmetric_migration   
  [####################] 100%  |  0:33:23  |  symmetric_migration/symmetric_migration   


In [584]:
res_dict["asymmetric_migration"]["no_migration"]
res_dict["asymmetric_migration"]["asymmetric_migration"]
#res_dict["symmetric_migration"]["no_migration"]
res_dict["no_migration"]["no_migration"]
#res_dict["no_migration"]["symmetric_migration"]["parameters"][1]

,fun,jac,kl_divergence,log_likelihood,message,nfev,nit,parameters,status,success,x
0,0.050932,[-4.9400055099086546e-14],0.050932,-6217.879747,Converged (|f_n-f_(n-1)| ~= 0),15.0,5.0,{'tdiv': 207812.4669740607},1.0,1.0,[207812.4669740607]
1,0.056832,[-1.5941820301899867e-10],0.056832,-6536.965831,Local minimum reached (|pg| ~= 0),14.0,4.0,{'tdiv': 194127.098247352},0.0,1.0,[194127.098247352]
2,0.047111,[-3.688396697440477e-15],0.047111,-6200.531366,Converged (|f_n-f_(n-1)| ~= 0),13.0,4.0,{'tdiv': 195961.44600195438},1.0,1.0,[195961.44600195438]
3,0.051390,[-2.1960968638219668e-14],0.051390,-6016.619226,Converged (|f_n-f_(n-1)| ~= 0),12.0,4.0,{'tdiv': 210044.85760750948},1.0,1.0,[210044.85760750948]
4,0.050849,[-2.488357827122459e-12],0.050849,-6285.086090,Local minimum reached (|pg| ~= 0),12.0,4.0,{'tdiv': 206175.80195611055},0.0,1.0,[206175.80195611055]


## Perform model selection with AIC


In [567]:
for model in models:
    AICs = []
    for testmodel in models:
        print(model, testmodel)
        lik = res_dict[model][testmodel]["log_likelihood"]
        nparams = len(res_dict[model][testmodel]["parameters"][0])
        aics = 2*nparams - 2*lik
        #print(aics, lik)
        AICs.append(np.min(aics))
    minv = np.min(AICs)
    delta_aic = np.array(AICs) - minv
    print("Delta AIC per model: ", delta_aic)
    print("AIC weight per model: ", np.exp(-0.5 * delta_aic))
    print("")

no_migration no_migration
no_migration asymmetric_migration
no_migration symmetric_migration
Delta AIC per model:  [0.57898907 3.98637385 0.        ]
AIC weight per model:  [0.74864188 0.13626048 1.        ]

asymmetric_migration no_migration
asymmetric_migration asymmetric_migration
asymmetric_migration symmetric_migration
Delta AIC per model:  [65.31044955  0.         69.31044955]
AIC weight per model:  [6.57682180e-15 1.00000000e+00 8.90076041e-16]

symmetric_migration no_migration
symmetric_migration asymmetric_migration
symmetric_migration symmetric_migration
Delta AIC per model:  [0.         6.88782933 4.00009967]
AIC weight per model:  [1.         0.03193941 0.13532854]



## Crap below here

In [548]:
sfs = sfs_dict["no_migration"][0]

91

In [592]:
!python -m momi.extract_sfs tmp_sfs.gz 10 simout/tmp_allele_counts.gz
sfs = momi.Sfs.load("tmp_sfs.gz")
print("Avg pairwise heterozygosity", sfs.avg_pairwise_hets[:5])
print("populations", sfs.populations)
print("percent missing data per population", sfs.p_missing)


Avg pairwise heterozygosity [[0.23809524 0.        ]
 [0.28571429 0.5952381 ]
 [0.21428571 0.21428571]
 [0.         1.10714286]
 [0.         0.75      ]]
populations ('pop1', 'pop2')
percent missing data per population [0.26984127 0.192     ]


In [190]:
print(f)
allele_counts = momi.SnpAlleleCounts.read_vcf(vcf_file=f, ind2pop=popdict, bed_file=f.split(".")[0]+".bed", ancestral_alleles=False)
allele_counts.dump(f.split(".")[0]+"_allele_counts.gz")
sfs = allele_counts.extract_sfs(n_blocks=None)
res = optimize(sfs)
print("percent missing data per population", sfs.p_missing, sfs.n_snps(), sfs.n_loci, res.parameters.tdiv)
subdict = {"pop1":6, "pop2":6}
allele_counts = allele_counts.down_sample(subdict)
sfs = allele_counts.extract_sfs(n_blocks=None)
res = optimize(sfs)
print("percent missing data per population", sfs.p_missing, sfs.n_snps(), sfs.n_loci, res.parameters.tdiv)

subdict = {"pop1":4, "pop2":4}
allele_counts = allele_counts.down_sample(subdict)
sfs = allele_counts.extract_sfs(n_blocks=None)
res = optimize(sfs)
print("percent missing data per population", sfs.p_missing, sfs.n_snps(), sfs.n_loci, res.parameters.tdiv)

subdict = {"pop1":3, "pop2":3}
allele_counts = allele_counts.down_sample(subdict)
sfs = allele_counts.extract_sfs(n_blocks=None)
res = optimize(sfs)
print("percent missing data per population", sfs.p_missing, sfs.n_snps(), sfs.n_loci, res.parameters.tdiv)

subdict = {"pop1":2, "pop2":2}
allele_counts = allele_counts.down_sample(subdict)
sfs = allele_counts.extract_sfs(n_blocks=None)
res = optimize(sfs)
print("percent missing data per population", sfs.p_missing, sfs.n_snps(), sfs.n_loci, res.parameters.tdiv)



simout/nomigration-sim2.vcf.gz


/home/isaac/miniconda2/envs/momi-py36/lib/python3.6/site-packages/momi/compute_sfs.py:511: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  [slice(None)]],
/home/isaac/miniconda2/envs/momi-py36/lib/python3.6/site-packages/autograd/numpy/numpy_boxes.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  def __getitem__(A, idx): return A[idx]
/home/isaac/miniconda2/envs/momi-py36/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:597: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)

percent missing data per population [0.28494548 0.27241363] 2717.0 928 194095.84240640234
percent missing data per population [0.13303507 0.12346818] 2625.0 926 194340.95056568255
percent missing data per population [0.03833145 0.03034077] 2358.0 899 194479.44609429862
percent missing data per population [0.02972925 0.02272324] 2071.0 867 196573.04078462542
percent missing data per population [0.00491746 0.000693  ] 1717.0 796 192115.485457603


In [187]:
allele_counts
sfs.sampled_n

array([2, 2])

In [198]:
sfs = momi.Sfs.load("simout/tmp.sfs")
sfs.config_array

array([[[3, 1],
        [6, 0]],

       [[2, 2],
        [6, 0]],

       [[2, 2],
        [4, 2]],

       [[4, 0],
        [4, 4]],

       [[3, 1],
        [1, 7]],

       [[4, 2],
        [0, 2]],

       [[6, 0],
        [3, 1]],

       [[6, 0],
        [3, 5]],

       [[5, 1],
        [8, 0]],

       [[5, 1],
        [0, 8]],

       [[8, 0],
        [1, 3]],

       [[8, 0],
        [7, 1]],

       [[8, 0],
        [5, 3]],

       [[8, 0],
        [0, 8]]])

In [247]:
from scipy.stats import entropy
skewed_abunds = np.array([100, 10, 10, 8, 6, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1])
even_abunds = np.array([10] * 15)


def diversity_indices(abunds):
    print("0D = {}".format(len(abunds)))
    print("1D = {} / {}".format(entropy(abunds), np.exp(entropy(abunds))))
    proportions = abunds/np.sum(abunds)
    squared_proportions = proportions * proportions
    print("2D = {}".format(1/np.sum(squared_proportions)))

print("skewed")
diversity_indices(skewed_abunds)
print("even")
diversity_indices(even_abunds)
print("highly skewed")
diversity_indices([10, 1, 1, 1, 1, 1, 1, 1, 1])


skewed
0D = 15
1D = 1.3827545055689163 / 3.9858656094059413
2D = 2.176857585139319
even
0D = 15
1D = 2.70805020110221 / 15.0
2D = 15.0
highly skewed
0D = 9
1D = 1.6111578173439172 / 5.008606923972825
2D = 2.999999999999999


In [393]:
#print(res_dict["no_migration"]["no_migration"])
print(dict(res_dict["no_migration"]["no_migration"][0]))
print(pd.DataFrame.from_dict(res_dict["no_migration"]["no_migration"][0]))
df = pd.DataFrame.from_dict(res_dict["no_migration"]["no_migration"])
pd.concat([df, pd.Series(res_dict["no_migration"]["no_migration"][0])])

KeyError: 0

In [401]:
res_dict["no_migration"]["no_migration"].append(pd.Series(res), ignore_index=True)

,fun,jac,kl_divergence,log_likelihood,message,nfev,nit,parameters,status,success,x
0,0.347906,[-1.3436429009052591e-11],0.347906,-728.09226,Local minimum reached (|pg| ~= 0),10.0,3.0,{'tdiv': 171637.84925481293},0.0,1.0,[171637.84925481293]


In [593]:
sfs.to_dict(vector=True)

[{((5, 1), (8, 0)): 1.0, ((8, 0), (0, 8)): 1.0},
 {((2, 2), (4, 2)): 1.0, ((6, 0), (3, 1)): 1.0},
 {((3, 1), (6, 0)): 1.0, ((8, 0), (1, 3)): 1.0},
 {((4, 0), (4, 4)): 1.0, ((6, 0), (3, 5)): 1.0},
 {((6, 0), (3, 1)): 1.0, ((8, 0), (5, 3)): 1.0},
 {((4, 2), (0, 2)): 1.0},
 {((6, 0), (3, 5)): 1.0, ((5, 1), (0, 8)): 1.0},
 {((2, 2), (6, 0)): 1.0, ((3, 1), (1, 7)): 1.0},
 {((8, 0), (7, 1)): 1.0}]